## Demonstrations de la balistique

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import HTML

# Demo 1: trajectory of lauched bullet without including air resistance force

Below demonstrates the trajectory of the bullet without considering the air resistance force.

$\vec r = (tv_0\cos\alpha, -1/2gt^2+tv_0\sin\alpha)$

In [2]:
g=9.81

def parabole(v_0, alpha,): 
    alpharad=alpha*np.pi/180 # conversion of the angle
    t = np.linspace(0,10,1000) # define time interval
    vit=[0,0,v_0*np.cos(alpharad),v_0*np.sin(alpharad)] # initial velocity
    
    ax = plt.gca()
    ax.quiver(0,0,v_0*np.cos(alpharad),v_0*np.sin(alpharad),color='red',angles='xy', scale_units='xy', scale=1)
    # initial velocity vector

    sx = v_0*np.cos(alpharad)*t
    sy = -0.5*g*t**2+v_0*np.sin(alpharad)*t # the coordination of trajectory
    
    plt.plot(sx, sy)
    plt.axis([0, np.max(sx), 0, np.max(sy)+20])  # change the scale of axes
    plt.xlabel('Horizontal movement of the bullet')
    plt.ylabel('Vertical movement of the bullet')
    plt.title('Trajectory of the lauched bullet')
    plt.axes().set_aspect('equal')
    plt.show()
    
interact(parabole, v_0=(0,50,1), alpha=(0,90,1))  ;  

interactive(children=(IntSlider(value=25, description='v_0', max=50), IntSlider(value=45, description='alpha', max=90), Output()), _dom_classes=('widget-interact',))

# Demo 2: trajectory of lauched bullet including air resistance force

Below demonstrates the trajectory of the bullet when considering the air resistance force with adjustable $\vec v_0$, $\alpha$ and $\beta$ ($\beta$ is the proportionality between the air resistance coefficient and mass of bullet)

$\vec r= (\int_{0}^{t} v_0\cos\alpha e^{-\beta t} dt,\int_{0}^{t} \frac{-g}{\beta}+(v_0\sin\alpha+\frac{-g}{\beta}) e^{-\beta t}) dt $;


In [3]:

from scipy.integrate import cumtrapz

def parabole_air(v_0,alpha,beta):
    alpharad = alpha*np.pi/180
    t = np.linspace(0,15,1000)
    vit = [0,0,v_0*np.cos(alpharad),v_0*np.sin(alpharad)]
    
    ax = plt.gca()
    ax.quiver(0,0,v_0*np.cos(alpharad),v_0*np.sin(alpharad),color='red',angles='xy', scale_units='xy', scale=1)
    
    trajectory_x = cumtrapz(v_0*np.cos(alpharad)*np.exp(-beta*t),t)
    trajectory_y = cumtrapz(-g/beta+(v_0*np.sin(alpharad)+g/beta)*np.exp(-beta*t),t)
    
    plt.plot(trajectory_x,trajectory_y)
    plt.axis([0, np.max(trajectory_x)+10, 0, np.max(trajectory_y)+20])    
    plt.xlabel('Horizontal movement of the bullet')
    plt.ylabel('Verticle movement of the bullet')
    plt.title('Trajectory of the lauched bullet')
    plt.axes().set_aspect('equal')
    plt.show()

interact(parabole_air, v_0=(0,50,1), alpha=(0,90,1), beta=(0.00001,1,0.01))  ;  

interactive(children=(IntSlider(value=25, description='v_0', max=50), IntSlider(value=45, description='alpha', max=90), FloatSlider(value=0.49001, description='beta', max=1.0, min=1e-05, step=0.01), Output()), _dom_classes=('widget-interact',))

# Demo 3: trajectory of lauched bullet with vs. without air resistance force

Below demonstrates the trajectory of the bullet with vs. without considering air resistance force

$\vec r = (tv_0\cos\alpha, -1/2gt^2+tv_0\sin\alpha)$;(without considering air resistance force)

$\vec r= (\int_{0}^{t} v_0\cos\alpha e^{-\beta t} dt,\int_{0}^{t} \frac{-g}{\beta}+(v_0\sin\alpha+\frac{-g}{\beta}) e^{-\beta t}) dt $; (considering air resistance force)

In [10]:

def parabole_comparison(v_0,alpha,beta):
    alpharad = alpha*np.pi/180
    t = np.linspace(0,15,1000)
    vit = [0,0,v_0*np.cos(alpharad),v_0*np.sin(alpharad)]
    ax = plt.gca()
    ax.quiver(0,0,v_0*np.cos(alpharad),v_0*np.sin(alpharad),color='red',angles='xy', scale_units='xy', scale=1)
    
    trajectory_x_without = v_0*np.cos(alpharad)*t
    trajectory_y_without = -0.5*g*t**2+v_0*np.sin(alpharad)*t
    plt.plot(trajectory_x_without, trajectory_y_without,'g', label='trajectory without considering air resistance force')
    plt.legend()
    
    trajectory_x_with = cumtrapz(v_0*np.cos(alpharad)*np.exp(-beta*t),t)
    trajectory_y_with = cumtrapz(-g/beta+(v_0*np.sin(alpharad)+g/beta)*np.exp(-beta*t),t)
    plt.plot(trajectory_x_with,trajectory_y_with,'b', label='trajectory considers air resistance force')
    plt.legend()
    
    plt.axis([0, 300, 0, 150])
    plt.xlabel('Horizontal movement of the bullet')
    plt.ylabel('Verticle movement of the bullet')
    plt.title('Trajectory of the lauched bullet')
    plt.axes().set_aspect('equal')
    plt.show()
    
interact(parabole_comparison, v_0=(0,50,1), alpha=(0,90,1), beta=(0.00001,1,0.01))  ;  

interactive(children=(IntSlider(value=25, description='v_0', max=50), IntSlider(value=45, description='alpha', max=90), FloatSlider(value=0.49001, description='beta', max=1.0, min=1e-05, step=0.01), Output()), _dom_classes=('widget-interact',))

# Demo 4: velocities of lauched bullet with vs. without air resistance force

Below demonstrates the velocity of the bullet with vs. without considering air resistance

$\vec v = (v_0\cos\alpha,-gt+v_0\sin\alpha)$; (without considering air resistance force)

$\vec v = (v_0\cos\alpha e^{-\beta t},\frac{-g}{\beta}+(v_0\sin\alpha+\frac{g}{\beta}) e^{-\beta t})$;(considering air resistance force)

In [5]:

def velocity_comparison(v_0, alpha, beta):
    alpharad=alpha*np.pi/180
    t = np.linspace(0,15,1000)
    
    v_x_without = v_0*np.cos(alpharad)
    v_y_without = v_0*np.sin(alpharad)-g*t
    v_without = np.sqrt((v_0*np.cos(alpharad))**2+(v_0*np.sin(alpharad)-g*t)**2)
    
    v_x_with = v_0*np.cos(alpharad)*np.exp(-beta*t)
    v_y_with = -g/beta+(v_0*np.sin(alpharad)+g/beta)*np.exp(-beta*t)
    v_with = np.sqrt((v_0*np.cos(alpharad)*np.exp(-beta*t))**2+(-g/beta+(v_0*np.sin(alpharad)+g/beta)*np.exp(-beta*t))**2)
    
    plt.plot(t, v_x_without.repeat(1000), 'r', label='Vx without considering air resistance force')
    plt.plot(t, v_y_without, 'g', label='Vy without considering air resistance force')    
    plt.plot(t, v_without, 'b', label='V without considering air resistance force')
    plt.plot(t,v_x_with,'r--',label='Vx considers air resistance force')
    plt.plot(t,v_y_with,'g--',label='Vy considers air resistance force')
    plt.plot(t, v_with, 'b--', label='V considers air resistance force')
    plt.legend(loc=2, bbox_to_anchor=(1.05,1.0),borderaxespad = 0.) 
    
    plt.grid(True)
    plt.xlabel('Time') 
    plt.ylabel('Velocity') 
    plt.title('Velocity of the bullet') 
    plt.show()
    
interact(velocity_comparison, v_0=(0,50,1), alpha=(0,90,1), beta=(0.00001,1,0.1))  ;  

interactive(children=(IntSlider(value=25, description='v_0', max=50), IntSlider(value=45, description='alpha', max=90), FloatSlider(value=0.40001000000000003, description='beta', max=1.0, min=1e-05), Output()), _dom_classes=('widget-interact',))

# Demo 5: trajectories of lauched bullet on slopes

In [6]:

def parabole_slope(v_0,alpha,beta,theta):
    alpharad = alpha*np.pi/180
    thetarad = theta*np.pi/180
    t = np.linspace(0.00001,30,1000)
    vit = [0,0,v_0*np.cos(alpharad+thetarad),v_0*np.sin(alpharad+thetarad)]
    ax = plt.gca()
    ax.quiver(0,0,v_0*np.cos(alpharad+thetarad),v_0*np.sin(alpharad+thetarad),color='red',angles='xy', scale_units = 'xy', scale=1)
    plt.axis([0, 300, 0, 150])  
    
    trajectory_x_without = v_0*np.cos(alpharad+thetarad)*t
    trajectory_y_without = -0.5*g*t**2+v_0*np.sin(alpharad+thetarad)*t 
    
    trajectory_x_with = cumtrapz(v_0*np.cos(alpharad+thetarad)*np.exp(-beta*t),t)
    trajectory_y_with = cumtrapz(-g/beta+(v_0*np.sin(alpharad+thetarad)+g/beta)*np.exp(-beta*t),t)
    
    slope_height = trajectory_x_without*np.tan(thetarad)   
    
    x_without=[]
    y_without=[]
    for i in range(1000):
        if trajectory_y_without[i]/trajectory_x_without[i] >= np.tan(thetarad):
            x_without.append(trajectory_x_without[i])
            y_without.append(trajectory_y_without[i])
    
    x_with=[]
    y_with=[]
    for i in range(999):
        if trajectory_y_with[i]/trajectory_x_with[i] >= np.tan(thetarad):
            x_with.append(trajectory_x_with[i])
            y_with.append(trajectory_y_with[i])
            
    plt.plot(x_without, y_without, 'g', label='trajectory without considering air resistance force')
    plt.plot(x_with, y_with, 'g--', label='trajectory considers air resistance force') 
    plt.plot(trajectory_x_without, slope_height, 'b')
    plt.legend()
    
    plt.fill_between(trajectory_x_without, slope_height, interpolate=True, color='blue', alpha=0.5)
    
    plt.xlabel('Horizontal movement of the bullet')
    plt.ylabel('Verticle movement of the bullet')
    plt.title('Trajectory of the lauched bullet')
    plt.axes().set_aspect('equal')
    plt.show()

interact(parabole_slope, v_0=(0,50,1), alpha=(0,90,1), beta=(0.00001,1,0.01), theta=(0,90,1))  ;  

interactive(children=(IntSlider(value=25, description='v_0', max=50), IntSlider(value=45, description='alpha', max=90), FloatSlider(value=0.49001, description='beta', max=1.0, min=1e-05, step=0.01), IntSlider(value=45, description='theta', max=90), Output()), _dom_classes=('widget-interact',))